<a href="https://colab.research.google.com/github/asungii/SAE-experiments/blob/main/EleutherAI_SAE_Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
%cd /content/gdrive/MyDrive/Colab Notebooks/sae

In [ ]:
! ls

In [ ]:
! pip install einops
! pip install jaxtyping
! pip install datasets
! pip install simple_parsing
! pip install accelerate

In [ ]:
! pip install multiprocessing

In [ ]:
! python -m sae

In [ ]:
! python -m sae EleutherAI/pythia-160m togethercomputer/RedPajama-Data-1T-Sample --attn_implementation=eager

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Map (num_proc=4): 100% 927262/930514 [40:09<00:30, 107.96 examples/s]

In [19]:
# For some reason I'm still getting the error even with the --attn_implementation="eager"
# oops i just didn't commit all the changes like an... idiot

with open('/content/gdrive/MyDrive/Colab Notebooks/sae/sae/__main__.py', 'r') as f:
    lines = f.readlines()
    for i in range(39, 80):
        print(lines[i])



    load_in_8bit: bool = False

    """Load the model in 8-bit mode."""

    

    data_preprocessing_num_proc: int = field(

        default_factory=lambda: cpu_count() // 2,

    )

    """Number of processes to use for preprocessing data"""



    attn_implementation: str = "sdpa"

    """Which implementation to use for attention in `transformers`. Pythia models require "eager"."""





def load_artifacts(args: RunConfig, rank: int) -> tuple[PreTrainedModel, Dataset]:

    if args.load_in_8bit:

        dtype = torch.float16

    elif torch.cuda.is_bf16_supported():

        dtype = torch.bfloat16

    else:

        dtype = "auto"



    model = AutoModelForCausalLM.from_pretrained(

        args.model,

        attn_implementation=args.attn_implementation,

        device_map={"": f"cuda:{rank}"},

        quantization_config=(

            BitsAndBytesConfig(load_in_8bit=args.load_in_8bit)

            if args.load_in_8bit

            else None

        ),

        torch_dtype

In [ ]:
# ykw maybe i'll try it programmatically

# okay that seemed to work

import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

from sae import SaeConfig, SaeTrainer, TrainConfig
from sae.data import chunk_and_tokenize

MODEL = "EleutherAI/pythia-160m"
dataset = load_dataset(
    "togethercomputer/RedPajama-Data-1T-Sample",
    split="train",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(MODEL, attn_implementation="eager")
tokenized = chunk_and_tokenize(dataset, tokenizer)


gpt = AutoModelForCausalLM.from_pretrained(
    MODEL,
    device_map={"": "cuda"},
    torch_dtype=torch.bfloat16,
)

cfg = TrainConfig(
    SaeConfig(gpt.config.hidden_size), batch_size=16
)
trainer = SaeTrainer(cfg, tokenized, gpt)

trainer.fit()